In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_validate
import pandas as pd
from sklearn.model_selection import GridSearchCV
from loguru import logger as lg
import multiprocessing
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC

import transformers
import torch
import random

import random
import string
import numpy as np
import unidecode
from unidecode import unidecode
from wordcloud import STOPWORDS
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score, f1_score
from torch.utils.data import TensorDataset
from tqdm.notebook import tqdm
from transformers import AutoModel, AutoTokenizer, BertModel, BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup, PreTrainedTokenizerFast
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader
from sklearn import metrics
import tensorflow as tf
from tqdm.notebook import tqdm
import numpy as np
import torch
transformers.logging.set_verbosity_debug()


In [38]:
NUM_PROCESSORS = (multiprocessing.cpu_count() - 2)
VERBOSE = 0
DEPARTMENTS_QUANTITY_TO_TRAIN = 6
lg.add("logging/finnlp_bert_binary_intra_class_experiment_2.log", rotation="1024 MB")
lg.info("\n\n####################################### NEW EXECUTION #######################################\n\n")
lg.info(f"Processors (cores) available: {NUM_PROCESSORS}")

2024-03-01 18:28:53.268 | INFO     | __main__:<module>:6 - 

####################################### NEW EXECUTION #######################################


2024-03-01 18:28:53.271 | INFO     | __main__:<module>:7 - Processors (cores) available: 38


In [9]:
brc = pd.read_csv("Banking_Regulation_Corpora_BRC_anonymous.csv" ,encoding="utf-8", low_memory=False, sep=";")
data_relevant = brc[brc['class'] == 1]
data_irrelevant = brc[brc['class'] == 0]

In [12]:
BATCH_SIZE = 31

<h3>Evaluate which regulators get in each department to make the intra-class balancing strategy</h3>

In [13]:
lt = ['DPT_25', 'DPT_3', 'DPT_2', 'DPT_9', 'DPT_5', 'DPT_16']

In [14]:
for l in lt:
    dfr = data_relevant[data_relevant.department.isin([l])]
    dddd = dict(dfr.regulatory_authority.value_counts())
    print(l,' - ',list(dict(sorted(dddd.items(), key=lambda item: item[1], reverse=True)).items())[:6])

DPT_25  -  [('BACEN', 666), ('PRESIDÊNCIA_DA_REPÚBLICA', 29), ('COAF', 28), ('DOU', 26), ('ANAC', 14), ('RFB', 14)]
DPT_3  -  [('B3', 302), ('BACEN', 175), ('CETIP', 99), ('ANBIMA', 55), ('CVM', 49), ('RFB', 7)]
DPT_2  -  [('CVM', 253), ('ANBIMA', 221), ('B3', 147), ('COAF', 18), ('BACEN', 16), ('RFB', 13)]
DPT_9  -  [('BACEN', 205), ('CIP', 198), ('DOU', 64), ('BNDES', 55), ('PRESIDÊNCIA_DA_REPÚBLICA', 32), ('NUCLEA', 17)]
DPT_5  -  [('BACEN', 193), ('CVM', 126), ('DOU', 107), ('RFB', 79), ('CFC', 22), ('CPC', 13)]
DPT_16  -  [('B3', 97), ('CETIP', 85), ('BACEN', 68), ('STN', 67), ('RFB', 24), ('ANBIMA', 20)]


In [15]:
# We chose the regulators with more documents in the relevant class of each of the three departments.
# The regulators must exist in both classes of the department.
# Only regulators with more than 10 documents in the relevant class where chosen.
# To make the undersampling, we took the regulator with less documents in the relevant class for each department, 26 documents at DPT_25, for example.

# DPT_25  -  [('BACEN', 666), ('PRESIDÊNCIA_DA_REPÚBLICA', 29), ('COAF', 28), ('DOU', 26)] # relevant
# DPT_25  -  [('DOU', 2982), ('BACEN', 467), ('COAF', 48), ('PRESIDÊNCIA_DA_REPÚBLICA', 32)] # irrelevant

# DPT_3  -  [('B3', 302), ('BACEN', 175), ('ANBIMA', 55), ('CVM', 49)] # relevant
# DPT_3  -  [('B3', 592), ('CVM', 368), ('BACEN', 257), ('ANBIMA', 85)] # irrelevant

# DPT_2  -  [('CVM', 253), ('ANBIMA', 221), ('B3', 147), ('RFB', 13)] # relevant
# DPT_2  -  [('B3', 1921), ('CVM', 783), ('RFB', 678), ('ANBIMA', 399)] # irrelevant

In [16]:
for l in lt:
    dfi = data_irrelevant[data_irrelevant.department.isin([l])]
    dddd = dict(dfi.regulatory_authority.value_counts())
    print(l,' - ',list(dict(sorted(dddd.items(), key=lambda item: item[1], reverse=True)).items())[:6])

DPT_25  -  [('DOU', 2982), ('BACEN', 467), ('ANAC', 421), ('COAF', 48), ('PRESIDÊNCIA_DA_REPÚBLICA', 32), ('NUCLEA', 26)]
DPT_3  -  [('B3', 592), ('CVM', 368), ('STN', 264), ('BACEN', 257), ('RFB', 161), ('ANBIMA', 85)]
DPT_2  -  [('B3', 1921), ('CVM', 783), ('CAMARA_MUNICIPAL_DO_RIO_DE_JANEIRO', 744), ('RFB', 678), ('ANBIMA', 399), ('SUSEP', 391)]
DPT_9  -  [('BACEN', 826), ('CIP', 712), ('CVM', 531), ('RFB', 331), ('STN', 269), ('DOU', 230)]
DPT_5  -  [('DOU', 1147), ('BACEN', 658), ('RFB', 327), ('PRESIDÊNCIA_DA_REPÚBLICA', 201), ('CVM', 158), ('NUCLEA', 60)]
DPT_16  -  [('BACEN', 337), ('STN', 294), ('RFB', 270), ('CVM', 162), ('B3', 121), ('CAMARA_MUNICIPAL_DO_RIO_DE_JANEIRO', 101)]


In [17]:
aux_relevant = []

SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_25 = 26
SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_3 = 49
SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_2 = 13

# DPT_25
dpt_25 = data_relevant[data_relevant.department.isin(['DPT_25'])]

regulator = dpt_25[dpt_25.regulatory_authority.isin(['BACEN'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_25]
aux_relevant.append(regulator)

regulator = dpt_25[dpt_25.regulatory_authority.isin(['PRESIDÊNCIA_DA_REPÚBLICA'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_25]
aux_relevant.append(regulator)

regulator = dpt_25[dpt_25.regulatory_authority.isin(['COAF'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_25]
aux_relevant.append(regulator)

regulator = dpt_25[dpt_25.regulatory_authority.isin(['DOU'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_25]
aux_relevant.append(regulator)
# DPT_25

# DPT_3
dpt_3 = data_relevant[data_relevant.department.isin(['DPT_3'])]

regulator = dpt_3[dpt_3.regulatory_authority.isin(['B3'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_3]
aux_relevant.append(regulator)

regulator = dpt_3[dpt_3.regulatory_authority.isin(['BACEN'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_3]
aux_relevant.append(regulator)

regulator = dpt_3[dpt_3.regulatory_authority.isin(['ANBIMA'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_3]
aux_relevant.append(regulator)

regulator = dpt_3[dpt_3.regulatory_authority.isin(['CVM'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_3]
aux_relevant.append(regulator)
# DPT_3

# DPT_2
dpt_2 = data_relevant[data_relevant.department.isin(['DPT_2'])]

regulator = dpt_2[dpt_2.regulatory_authority.isin(['CVM'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_2]
aux_relevant.append(regulator)

regulator = dpt_2[dpt_2.regulatory_authority.isin(['ANBIMA'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_2]
aux_relevant.append(regulator)

regulator = dpt_2[dpt_2.regulatory_authority.isin(['B3'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_2]
aux_relevant.append(regulator)

regulator = dpt_2[dpt_2.regulatory_authority.isin(['RFB'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_2]
aux_relevant.append(regulator)
# DPT_2

data_relevant = pd.concat(aux_relevant)

data_relevant[data_relevant.text_words < 50]

,class,department,entry_date,general_id,normative_identifier,publication_date,regulatory_authority,subject,subject_length,subject_unique_words,subject_words,text,text_length,text_unique_words,text_words,type,unique_document_id,url


In [18]:
data_relevant.shape

(352, 18)

In [19]:
(13*4)+(49*4)+(26*4)

352

In [20]:
aux_irrelevant = []

SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_25 = 26
SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_3 = 49
SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_2 = 13

# DPT_25
dpt_25 = data_irrelevant[data_irrelevant.department.isin(['DPT_25'])]

regulator = dpt_25[dpt_25.regulatory_authority.isin(['BACEN'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_25]
aux_irrelevant.append(regulator)

regulator = dpt_25[dpt_25.regulatory_authority.isin(['PRESIDÊNCIA_DA_REPÚBLICA'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_25]
aux_irrelevant.append(regulator)

regulator = dpt_25[dpt_25.regulatory_authority.isin(['COAF'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_25]
aux_irrelevant.append(regulator)

regulator = dpt_25[dpt_25.regulatory_authority.isin(['DOU'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_25]
aux_irrelevant.append(regulator)
# DPT_25

# DPT_3
dpt_3 = data_irrelevant[data_irrelevant.department.isin(['DPT_3'])]

regulator = dpt_3[dpt_3.regulatory_authority.isin(['B3'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_3]
aux_irrelevant.append(regulator)

regulator = dpt_3[dpt_3.regulatory_authority.isin(['BACEN'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_3]
aux_irrelevant.append(regulator)

regulator = dpt_3[dpt_3.regulatory_authority.isin(['ANBIMA'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_3]
aux_irrelevant.append(regulator)

regulator = dpt_3[dpt_3.regulatory_authority.isin(['CVM'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_3]
aux_irrelevant.append(regulator)
# DPT_3

# DPT_2
dpt_2 = data_irrelevant[data_irrelevant.department.isin(['DPT_2'])]

regulator = dpt_2[dpt_2.regulatory_authority.isin(['CVM'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_2]
aux_irrelevant.append(regulator)

regulator = dpt_2[dpt_2.regulatory_authority.isin(['ANBIMA'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_2]
aux_irrelevant.append(regulator)

regulator = dpt_2[dpt_2.regulatory_authority.isin(['B3'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_2]
aux_irrelevant.append(regulator)

regulator = dpt_2[dpt_2.regulatory_authority.isin(['RFB'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_2]
aux_irrelevant.append(regulator)
# DPT_2

data_irrelevant = pd.concat(aux_irrelevant)

data_irrelevant[data_irrelevant.text_words < 50]

,class,department,entry_date,general_id,normative_identifier,publication_date,regulatory_authority,subject,subject_length,subject_unique_words,subject_words,text,text_length,text_unique_words,text_words,type,unique_document_id,url


In [21]:
data_irrelevant.shape

(352, 18)

In [22]:
data_irrelevant.head(3)

,class,department,entry_date,general_id,normative_identifier,publication_date,regulatory_authority,subject,subject_length,subject_unique_words,subject_words,text,text_length,text_unique_words,text_words,type,unique_document_id,url
7846,0,DPT_25,2020-08-31,2169,0301/2020,2020-08-31,BACEN,Caracterizado fornecimento intempestivo ao Ba...,294,36.0,49.0,​DEPARTAMENTO DE RESOLUÇÃO E DE AÇÃO SANCIONAD...,758,81.0,113.0,PROCESSO ADMINISTRATIVO SANCIONADOR,729080,https://www.bcb.gov.br/estabilidadefinanceira/...
7847,0,DPT_25,2020-08-31,2170,36121,2020-08-31,BACEN,Divulga nome aprovado de pessoas eleitas/nomea...,167,21.0,23.0,Divulgamos o nome aprovado de pessoas eleitas/...,7080,407.0,966.0,COMUNICADO,729131,https://www.bcb.gov.br/estabilidadefinanceira/...
7848,0,DPT_25,2020-08-31,2171,36123,2020-08-31,BACEN,Divulga nome de pessoas com intenção de ocupar...,140,19.0,21.0,Divulgamos os nomes de pretendentes a cargos d...,1558,149.0,276.0,COMUNICADO,729151,https://www.bcb.gov.br/estabilidadefinanceira/...


In [23]:
data_relevant.count()

class                   352
department              352
entry_date              352
general_id              352
normative_identifier    352
publication_date        352
regulatory_authority    352
subject                 352
subject_length          352
subject_unique_words    352
subject_words           352
text                    352
text_length             352
text_unique_words       352
text_words              352
type                    352
unique_document_id      352
url                     352
dtype: int64

In [24]:
data_relevant = data_relevant[data_relevant.text_words >= 50]
data_relevant.count()

class                   352
department              352
entry_date              352
general_id              352
normative_identifier    352
publication_date        352
regulatory_authority    352
subject                 352
subject_length          352
subject_unique_words    352
subject_words           352
text                    352
text_length             352
text_unique_words       352
text_words              352
type                    352
unique_document_id      352
url                     352
dtype: int64

In [25]:
data_irrelevant.count()

class                   352
department              352
entry_date              352
general_id              352
normative_identifier    352
publication_date        352
regulatory_authority    352
subject                 352
subject_length          352
subject_unique_words    352
subject_words           352
text                    352
text_length             352
text_unique_words       352
text_words              352
type                    352
unique_document_id      352
url                     352
dtype: int64

In [26]:
data_irrelevant = data_irrelevant[data_irrelevant.text_words >= 50]
data_irrelevant.count()

class                   352
department              352
entry_date              352
general_id              352
normative_identifier    352
publication_date        352
regulatory_authority    352
subject                 352
subject_length          352
subject_unique_words    352
subject_words           352
text                    352
text_length             352
text_unique_words       352
text_words              352
type                    352
unique_document_id      352
url                     352
dtype: int64

In [27]:
data_relevant['department'].unique()

array(['DPT_25', 'DPT_3', 'DPT_2'], dtype=object)

In [29]:
len(list(data_relevant['department'].unique()))

3

In [30]:
class Model_BERT():
    
    def model_path_def(self, name_model):
        if name_model == "BERTIMBAU_BASE":
            model_path = MODEL_PATH
        elif name_model == "XXXX":
            model_path = MODEL_PATH_NEW
        else:
            print("Nome de modelo inválido")
            model_path = "None"
            
        return model_path

    def tokenizer_model(self, model_path, df, token):
        tokenizer = BertTokenizer.from_pretrained(BERTIMBAU_TOKENIZER, do_lower_case=True)

        # Tokenizing the training data
        encoded_data_train = tokenizer.batch_encode_plus(
            df[df.data_type=='train'].new_content.tolist(), 
            add_special_tokens=True, 
            return_attention_mask=True, 
            padding=True, 
            max_length=token, 
            return_tensors='pt',
            truncation=True
        )

        # Tokenizing test data
        encoded_data_val = tokenizer.batch_encode_plus(
            df[df.data_type=='val'].new_content.tolist(), 
            add_special_tokens=True, 
            return_attention_mask=True, 
            padding=True,
            max_length=token, 
            return_tensors='pt',
            truncation=True
        )


        input_ids_train = encoded_data_train['input_ids']
        attention_masks_train = encoded_data_train['attention_mask']
        labels_train = torch.tensor(df[df.data_type=='train'].label_num.values)

        input_ids_val = encoded_data_val['input_ids']
        attention_masks_val = encoded_data_val['attention_mask']
        labels_val = torch.tensor(df[df.data_type=='val'].label_num.values)

        dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
        dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)
        
        return(dataset_train, dataset_val)

    def setup_model(self, model_path, dataset_train, dataset_val, epochs):
        model = BertForSequenceClassification.from_pretrained(model_path,
                                                              num_labels=len(LABEL_DICT),
                                                              output_attentions=False,
                                                              output_hidden_states=False)

        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model.to(device)
        batch_size = BATCH_SIZE
        dataloader_train = DataLoader(dataset_train, 
                                      sampler=RandomSampler(dataset_train), 
                                      batch_size=batch_size)
        dataloader_validation = DataLoader(dataset_val, 
                                           sampler=SequentialSampler(dataset_val), 
                                           batch_size=batch_size)
        optimizer = AdamW(model.parameters(),
                          lr=1e-5, 
                          eps=1e-8)

        scheduler = get_linear_schedule_with_warmup(optimizer, 
                                                    num_warmup_steps=100,
                                                    num_training_steps=len(dataloader_train)*epochs)
        return(model, dataloader_train, dataloader_validation, epochs, scheduler, optimizer, device)


    def f1_score_func(self, preds, labels):
        preds_flat = np.argmax(preds, axis=1).flatten()
        labels_flat = labels.flatten()

        scr = f1_score(labels_flat, preds_flat, average='binary')
        lg.info(f"f1_score_binary: {scr}")
        return scr

    def accuracy_per_class(self, preds, labels):
        label_dict_inverse = {v: k for k, v in LABEL_DICT.items()}

        preds_flat = np.argmax(preds, axis=1).flatten()
        labels_flat = labels.flatten()

        for label in np.unique(labels_flat):
            y_preds = preds_flat[labels_flat==label]
            y_true = labels_flat[labels_flat==label]
            print(f'Class: {label_dict_inverse[label]}')
            print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}')
            print(f'% Accuracy: {len(y_preds[y_preds==label])/len(y_true)}\n')
            lg.info(f'Class: {label_dict_inverse[label]}')
            lg.info(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}')
            lg.info(f'% Accuracy: {len(y_preds[y_preds==label])/len(y_true)}\n')
        
    def evaluate(self, model, dataloader_val, device):

        model.eval()

        loss_val_total = 0
        predictions, true_vals = [], []

        for batch in dataloader_val:

            batch = tuple(b.to(device) for b in batch)

            inputs = {'input_ids':      batch[0],
                      'attention_mask': batch[1],
                      'labels':         batch[2],
                     }

            with torch.no_grad():        
                outputs = model(**inputs)

            loss = outputs[0]
            logits = outputs[1]
            loss_val_total += loss.item()

            logits = logits.detach().cpu().numpy()
            label_ids = inputs['labels'].cpu().numpy()
            predictions.append(logits)
            true_vals.append(label_ids)

        loss_val_avg = loss_val_total/len(dataloader_val) 

        predictions = np.concatenate(predictions, axis=0)
        true_vals = np.concatenate(true_vals, axis=0)

        lg.info(f"loss_val_avg, predictions, true_vals: {loss_val_avg}, {predictions}, {true_vals}")

        return loss_val_avg, predictions, true_vals

    def results(self, model_path, model_name, epochs, dataloader_validation, device, token):
        model = BertForSequenceClassification.from_pretrained(model_path,
                                                              num_labels=len(LABEL_DICT),
                                                              output_attentions=False,
                                                              output_hidden_states=False)

        model.to(device)

        model.load_state_dict(torch.load(f'{MODEL_SAVE_PATH}/token{token}_model_data_{model_name}_epoch_{epochs}.model', map_location=torch.device('cpu')))

        _, predictions, true_vals = Model_BERT().evaluate(model, dataloader_validation, device)
        Model_BERT().accuracy_per_class(predictions, true_vals)
        from sklearn import metrics
        preds_flat = np.argmax(predictions, axis=1).flatten()
        metrics_report = metrics.classification_report(true_vals,preds_flat, output_dict=True)
        lg.info(metrics_report)
        
        return metrics_report
    

    def training_model(self, model, model_name, dataloader_train, dataloader_validation, epochs, scheduler, optimizer, device, token):
        seed_val = 1
        random.seed(seed_val)
        np.random.seed(seed_val)
        torch.manual_seed(seed_val)
        torch.cuda.manual_seed_all(seed_val)
        epoch_array = []
        f1_array = []
        train_loss_array = []
        val_loss_array = []

        for epoch in tqdm(range(1, epochs+1)):

            model.train()

            loss_train_total = 0

            progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
            for batch in progress_bar:

                model.zero_grad()

                batch = tuple(b.to(device) for b in batch)

                inputs = {'input_ids':      batch[0],
                          'attention_mask': batch[1],
                          'labels':         batch[2],
                         }       

                outputs = model(**inputs)

                loss = outputs[0]
                loss_train_total += loss.item()
                loss.backward()

                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

                optimizer.step()
                scheduler.step()

                progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})


            torch.save(model.state_dict(),f'{MODEL_SAVE_PATH}/token{token}_model_data_{model_name}_epoch_{epoch}.model')
            model.save_pretrained(f"{MODEL_SAVE_PATH}/token{token}_model_data_{model_name}_epoch_{epoch}")
            tqdm.write(f'\nEpoch {epoch}')
            print(f'\nEpoch {epoch}')
            lg.info(f'\nEpoch {epoch}')

            loss_train_avg = loss_train_total/len(dataloader_train)            
            tqdm.write(f'Training loss: {loss_train_avg}')
            lg.info(f'Training loss: {loss_train_avg}')

            val_loss, predictions, true_vals = Model_BERT().evaluate(model, dataloader_validation, device)
            val_f1 = Model_BERT().f1_score_func(predictions, true_vals)
            tqdm.write(f'Validation loss: {val_loss}')
            print(f'Validation loss: {val_loss}')
            lg.info(f'Validation loss: {val_loss}')
            
            tqdm.write(f'F1 Score (Weighted/Binary): {val_f1}')
            print(f'F1 Score (Weighted/Binary): {val_f1}')
            lg.info(f'F1 Score (Weighted/Binary): {val_f1}')
            
            epoch_array.append(epoch)
            f1_array.append(val_f1)
            val_loss_array.append(val_loss)
            train_loss_array.append(loss_train_avg)

        return(epoch_array, f1_array, val_loss_array, train_loss_array)
    
    def read_df(self, df):
        df['label_num'] = df.label.replace(LABEL_DICT)

        X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label_num.values, 
                                                  test_size=0.4, 
                                                  random_state=42, 
                                                  stratify=df.label.values)
        df['data_type'] = ['not_set']*df.shape[0]
        df.loc[X_train, 'data_type'] = 'train'
        df.loc[X_val, 'data_type'] = 'val'
        
        return(df)

    def call_setup(self, model_name, df, token, epochs):
        df = Model_BERT().read_df(df)
        model_path = Model_BERT().model_path_def(model_name)
        print(model_path)
        dataset_train, dataset_val = Model_BERT().tokenizer_model(model_path, df, token)
        model, dataloader_train, dataloader_val, epochs, scheduler, optimizer, device = Model_BERT().setup_model(model_path, dataset_train, dataset_val, epochs)
        epoch_array, f1_array, val_loss_array, train_loss_array = Model_BERT().training_model(model, model_name, dataloader_train, dataloader_val, epochs, scheduler, optimizer, device, token)
        df_performa = pd.DataFrame()
        df_performa["epoch"] = epoch_array
        df_performa["f1"] = f1_array
        df_performa["val_loss"] = val_loss_array
        df_performa["train_loss"] = train_loss_array
        metrics_report = Model_BERT().results(model_path,model_name,epochs,dataloader_val, device, token)
        lg.info(f'Metric Report Call Setup: {metrics_report}')
        
        return metrics_report, df_performa

In [31]:
def cleaning(df,column):
    # Remove urls
    df.loc[:,column] = df[column].replace(r'https?:\/\/.*?[\s+]', ' ', regex=True)
    # Remove urls without https?
    df.loc[:,column] = df[column].replace(r'www.*?[\s+]', ' ', regex=True)
    # Remove email address
    df.loc[:,column] = df[column].replace(r'(?P<email_address>[\w\.-]+@[\w\.-]+\.[\w]+)', ' ', regex=True)
    
    df.loc[:,column] = df[column].str.replace('º',' ')
    df.loc[:,column] = df[column].str.replace('ª',' ')

    df.loc[:,column] = df[column].str.replace('1',' ')
    df.loc[:,column] = df[column].str.replace('2',' ')
    df.loc[:,column] = df[column].str.replace('3',' ')
    df.loc[:,column] = df[column].str.replace('4',' ')
    df.loc[:,column] = df[column].str.replace('5',' ')
    df.loc[:,column] = df[column].str.replace('6',' ')
    df.loc[:,column] = df[column].str.replace('7',' ')
    df.loc[:,column] = df[column].str.replace('8',' ')
    df.loc[:,column] = df[column].str.replace('9',' ')
    df.loc[:,column] = df[column].str.replace('0',' ')

    df.loc[:,column] = df[column].str.replace('/',' ')

    df.loc[:,column] = df[column].str.replace('\r',' ')
    df.loc[:,column] = df[column].str.replace('\n',' ')
    df.loc[:,column] = df[column].str.replace('\t',' ')
    df.loc[:,column] = df[column].str.replace('\\',' ')

    df.loc[:,column] = df[column].str.replace('-',' ')

    df.loc[:,column] = df[column].str.replace('–',' ')
    df.loc[:,column] = df[column].str.replace('“',' ')
    df.loc[:,column] = df[column].str.replace('”',' ')
    df.loc[:,column] = df[column].str.replace('’',' ')
    df.loc[:,column] = df[column].str.replace('_',' ')
    df.loc[:,column] = df[column].str.replace('.',' ')
    df.loc[:,column] = df[column].str.replace(',',' ')
    df.loc[:,column] = df[column].str.replace('|',' ')
    df.loc[:,column] = df[column].str.replace('=',' ')
    df.loc[:,column] = df[column].str.replace('@',' ')
    df.loc[:,column] = df[column].str.replace('$',' ')

    df.loc[:,column] = df[column].str.replace('°',' ')
    df.loc[:,column] = df[column].str.replace('§',' ')
    df.loc[:,column] = df[column].str.replace('•',' ')
    df.loc[:,column] = df[column].str.replace('▪',' ')

    df.loc[:,column] = df[column].str.replace('%',' ')
    df.loc[:,column] = df[column].str.replace('&',' ')
    df.loc[:,column] = df[column].str.replace('*',' ')
    df.loc[:,column] = df[column].str.replace('+',' ')
    df.loc[:,column] = df[column].str.replace(':',' ')
    df.loc[:,column] = df[column].str.replace(';',' ')

    df.loc[:,column] = df[column].str.replace('!',' ')
    df.loc[:,column] = df[column].str.replace('?',' ')
    df.loc[:,column] = df[column].str.replace('#',' ')
    df.loc[:,column] = df[column].str.replace('\'',' ')
    df.loc[:,column] = df[column].str.replace('"',' ')

    df.loc[:,column] = df[column].str.replace('<',' ')
    df.loc[:,column] = df[column].str.replace('>',' ')

    df.loc[:,column] = df[column].str.replace('(',' ')
    df.loc[:,column] = df[column].str.replace(')',' ')

    df.loc[:,column] = df[column].str.replace('{',' ')
    df.loc[:,column] = df[column].str.replace('}',' ')

    df.loc[:,column] = df[column].str.replace('[',' ')
    df.loc[:,column] = df[column].str.replace(']',' ')

    # Remove the same character repeated more than twice
    df.loc[:,column] = df[column].replace(r'([a-z])\1{2,}', ' ', regex=True)
    
    # To replace more than one white space by only one white space
    df.loc[:,column] = df[column].replace(r'\s+', ' ', regex=True)

    return df

<h3>Selecting the Departments</h3>

In [32]:
boards = data_relevant['department']
# Getting the 6 most populated boards
boards_list = list(boards.unique())[:DEPARTMENTS_QUANTITY_TO_TRAIN]
lg.info(f"boards_list: {boards_list}")

2024-03-01 16:33:21.915 | INFO     | __main__:<module>:4 - boards_list: ['DPT_25', 'DPT_3', 'DPT_2']


In [33]:
df_modelagem_init = pd.concat([data_irrelevant, data_relevant])

In [34]:
df_modelagem_init['class'] = df_modelagem_init['class'].astype(str)

In [35]:
df_modelagem_init = df_modelagem_init.rename(columns={"class":"label"})

In [36]:
df_modelagem_init.count()

label                   704
department              704
entry_date              704
general_id              704
normative_identifier    704
publication_date        704
regulatory_authority    704
subject                 704
subject_length          704
subject_unique_words    704
subject_words           704
text                    704
text_length             704
text_unique_words       704
text_words              704
type                    704
unique_document_id      704
url                     704
dtype: int64

In [ ]:
for department in boards_list:
    lg.debug(f"Department: {department}")
    df_modelagem = df_modelagem_init[df_modelagem_init.department == department]

    df_modelagem = df_modelagem.dropna()
    df_modelagem = df_modelagem.reset_index(drop=True)
    df_modelagem = df_modelagem.rename(columns={"text":"content"})

    df_modelagem = cleaning(df_modelagem,"content")

    new_content = []
    for texto in df_modelagem.content:
        new_content.append(texto.replace("[BUSINESS NEED]", ""))
    df_modelagem["new_content"] = new_content
    
    LABEL_DICT = {'0': 0, '1': 1}

    MODEL_SAVE_PATH = "notebooks/testes/scoring"
    BERTIMBAU_BASE = "datasets/BERTIMBAU_BASE"
    BERTIMBAU_TOKENIZER = "tokenizer/BERTimbau_Tokenizer"

    MODEL_PATH = BERTIMBAU_BASE
    MODEL_PATH_NEW = BERTIMBAU_BASE

    tokenizer = BertTokenizer.from_pretrained(BERTIMBAU_TOKENIZER, do_lower_case=True)

    lg.debug(f"Department count samples: {df_modelagem.count()}")
    lg.debug(f"Department count samples class 1: {df_modelagem[df_modelagem.label == '1'].count()}")
    lg.debug(f"Department count samples class 0: {df_modelagem[df_modelagem.label == '0'].count()}")
    

    Model_BERT().call_setup("BERTIMBAU_BASE", df_modelagem, 512, 5)

<h2>The output result is in the next cell. It was copied from the log file.</h2>

In [ ]:
# 2023-10-10 10:30:47.737 | INFO     | __main__:<module>:5 - 

# ####################################### NEW EXECUTION #######################################


# 2023-10-10 10:30:47.738 | INFO     | __main__:<module>:6 - Processors (cores) available: 38
# 2023-10-10 10:30:57.737 | INFO     | __main__:<module>:4 - boards_list: ['DPT_25', 'DPT_3', 'DPT_2']
# 2023-10-10 10:30:57.788 | DEBUG    | __main__:<module>:2 - Department: DPT_25
# 2023-10-10 10:30:58.299 | DEBUG    | __main__:<module>:37 - Department count samples: label                   208
# department              208
# entry_date              208
# general_id              208
# normative_identifier    208
# publication_date        208
# regulatory_authority    208
# subject                 208
# subject_length          208
# subject_unique_words    208
# subject_words           208
# content                 208
# text_length             208
# text_unique_words       208
# text_words              208
# type                    208
# unique_document_id      208
# url                     208
# new_content             208
# dtype: int64
# 2023-10-10 10:30:58.303 | DEBUG    | __main__:<module>:38 - Department count samples class 1: label                   104
# department              104
# entry_date              104
# general_id              104
# normative_identifier    104
# publication_date        104
# regulatory_authority    104
# subject                 104
# subject_length          104
# subject_unique_words    104
# subject_words           104
# content                 104
# text_length             104
# text_unique_words       104
# text_words              104
# type                    104
# unique_document_id      104
# url                     104
# new_content             104
# dtype: int64
# 2023-10-10 10:30:58.305 | DEBUG    | __main__:<module>:39 - Department count samples class 0: label                   104
# department              104
# entry_date              104
# general_id              104
# normative_identifier    104
# publication_date        104
# regulatory_authority    104
# subject                 104
# subject_length          104
# subject_unique_words    104
# subject_words           104
# content                 104
# text_length             104
# text_unique_words       104
# text_words              104
# type                    104
# unique_document_id      104
# url                     104
# new_content             104
# dtype: int64
# 2023-10-10 10:31:20.724 | INFO     | __main__:training_model:210 - 
# Epoch 1
# 2023-10-10 10:31:20.725 | INFO     | __main__:training_model:214 - Training loss: 0.6965916454792023
# 2023-10-10 10:31:21.485 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.6993502577145895, [[0.19235902 0.11708909]

# 2023-10-10 10:31:21.489 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.16949152542372883
# 2023-10-10 10:31:21.490 | INFO     | __main__:training_model:220 - Validation loss: 0.6993502577145895
# 2023-10-10 10:31:21.491 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.16949152542372883
# 2023-10-10 10:31:27.066 | INFO     | __main__:training_model:210 - 
# Epoch 2
# 2023-10-10 10:31:27.068 | INFO     | __main__:training_model:214 - Training loss: 0.6985060423612595
# 2023-10-10 10:31:27.830 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.6988730033238729, [[0.18783389 0.11924584]

# 2023-10-10 10:31:27.833 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.19354838709677416
# 2023-10-10 10:31:27.834 | INFO     | __main__:training_model:220 - Validation loss: 0.6988730033238729
# 2023-10-10 10:31:27.835 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.19354838709677416
# 2023-10-10 10:31:33.383 | INFO     | __main__:training_model:210 - 
# Epoch 3
# 2023-10-10 10:31:33.385 | INFO     | __main__:training_model:214 - Training loss: 0.6937734484672546
# 2023-10-10 10:31:34.148 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.698234498500824, [[ 0.17943902  0.12156092]

# 2023-10-10 10:31:34.150 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.2222222222222222
# 2023-10-10 10:31:34.152 | INFO     | __main__:training_model:220 - Validation loss: 0.698234498500824
# 2023-10-10 10:31:34.153 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.2222222222222222
# 2023-10-10 10:31:39.681 | INFO     | __main__:training_model:210 - 
# Epoch 4
# 2023-10-10 10:31:39.682 | INFO     | __main__:training_model:214 - Training loss: 0.6967955827713013
# 2023-10-10 10:31:40.448 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.6974149346351624, [[ 0.16675785  0.12218992]

# 2023-10-10 10:31:40.452 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.2985074626865672
# 2023-10-10 10:31:40.453 | INFO     | __main__:training_model:220 - Validation loss: 0.6974149346351624
# 2023-10-10 10:31:40.453 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.2985074626865672
# 2023-10-10 10:31:46.255 | INFO     | __main__:training_model:210 - 
# Epoch 5
# 2023-10-10 10:31:46.257 | INFO     | __main__:training_model:214 - Training loss: 0.7008830606937408
# 2023-10-10 10:31:47.025 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.6961301763852438, [[ 0.14382999  0.12268669]

# 2023-10-10 10:31:47.028 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.37500000000000006
# 2023-10-10 10:31:47.030 | INFO     | __main__:training_model:220 - Validation loss: 0.6961301763852438
# 2023-10-10 10:31:47.031 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.37500000000000006
# 2023-10-10 10:31:49.363 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.6961301763852438, [[ 0.14382999  0.12268669]
 
# 2023-10-10 10:31:49.365 | INFO     | __main__:accuracy_per_class:101 - Class: 0
# 2023-10-10 10:31:49.366 | INFO     | __main__:accuracy_per_class:102 - Accuracy: 19/42
# 2023-10-10 10:31:49.367 | INFO     | __main__:accuracy_per_class:103 - % Accuracy: 0.4523809523809524

# 2023-10-10 10:31:49.367 | INFO     | __main__:accuracy_per_class:101 - Class: 1
# 2023-10-10 10:31:49.368 | INFO     | __main__:accuracy_per_class:102 - Accuracy: 15/42
# 2023-10-10 10:31:49.369 | INFO     | __main__:accuracy_per_class:103 - % Accuracy: 0.35714285714285715

# 2023-10-10 10:31:49.376 | INFO     | __main__:results:157 - {'0': {'precision': 0.41304347826086957, 'recall': 0.4523809523809524, 'f1-score': 0.4318181818181818, 'support': 42}, '1': {'precision': 0.39473684210526316, 'recall': 0.35714285714285715, 'f1-score': 0.37500000000000006, 'support': 42}, 'accuracy': 0.40476190476190477, 'macro avg': {'precision': 0.40389016018306634, 'recall': 0.40476190476190477, 'f1-score': 0.40340909090909094, 'support': 84}, 'weighted avg': {'precision': 0.40389016018306634, 'recall': 0.40476190476190477, 'f1-score': 0.40340909090909094, 'support': 84}}
# 2023-10-10 10:31:49.378 | INFO     | __main__:call_setup:260 - Metric Report Call Setup: {'0': {'precision': 0.41304347826086957, 'recall': 0.4523809523809524, 'f1-score': 0.4318181818181818, 'support': 42}, '1': {'precision': 0.39473684210526316, 'recall': 0.35714285714285715, 'f1-score': 0.37500000000000006, 'support': 42}, 'accuracy': 0.40476190476190477, 'macro avg': {'precision': 0.40389016018306634, 'recall': 0.40476190476190477, 'f1-score': 0.40340909090909094, 'support': 84}, 'weighted avg': {'precision': 0.40389016018306634, 'recall': 0.40476190476190477, 'f1-score': 0.40340909090909094, 'support': 84}}
# 2023-10-10 10:31:49.381 | DEBUG    | __main__:<module>:2 - Department: DPT_3
# 2023-10-10 10:31:52.283 | DEBUG    | __main__:<module>:37 - Department count samples: label                   392
# department              392
# entry_date              392
# general_id              392
# normative_identifier    392
# publication_date        392
# regulatory_authority    392
# subject                 392
# subject_length          392
# subject_unique_words    392
# subject_words           392
# content                 392
# text_length             392
# text_unique_words       392
# text_words              392
# type                    392
# unique_document_id      392
# url                     392
# new_content             392
# dtype: int64
# 2023-10-10 10:31:52.286 | DEBUG    | __main__:<module>:38 - Department count samples class 1: label                   196
# department              196
# entry_date              196
# general_id              196
# normative_identifier    196
# publication_date        196
# regulatory_authority    196
# subject                 196
# subject_length          196
# subject_unique_words    196
# subject_words           196
# content                 196
# text_length             196
# text_unique_words       196
# text_words              196
# type                    196
# unique_document_id      196
# url                     196
# new_content             196
# dtype: int64
# 2023-10-10 10:31:52.290 | DEBUG    | __main__:<module>:39 - Department count samples class 0: label                   196
# department              196
# entry_date              196
# general_id              196
# normative_identifier    196
# publication_date        196
# regulatory_authority    196
# subject                 196
# subject_length          196
# subject_unique_words    196
# subject_words           196
# content                 196
# text_length             196
# text_unique_words       196
# text_words              196
# type                    196
# unique_document_id      196
# url                     196
# new_content             196
# dtype: int64
# 2023-10-10 10:32:48.337 | INFO     | __main__:training_model:210 - 
# Epoch 1
# 2023-10-10 10:32:48.339 | INFO     | __main__:training_model:214 - Training loss: 0.6908790990710258
# 2023-10-10 10:32:49.758 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.7044742604096731, [[ 0.15235686  0.08747568]
 
# 2023-10-10 10:32:49.762 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.09411764705882351
# 2023-10-10 10:32:49.763 | INFO     | __main__:training_model:220 - Validation loss: 0.7044742604096731
# 2023-10-10 10:32:49.765 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.09411764705882351
# 2023-10-10 10:32:58.467 | INFO     | __main__:training_model:210 - 
# Epoch 2
# 2023-10-10 10:32:58.470 | INFO     | __main__:training_model:214 - Training loss: 0.6866002678871155
# 2023-10-10 10:32:59.896 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.6963733931382498, [[ 0.13420327  0.1274321 ]
 
# 2023-10-10 10:32:59.900 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.35714285714285715
# 2023-10-10 10:32:59.901 | INFO     | __main__:training_model:220 - Validation loss: 0.6963733931382498
# 2023-10-10 10:32:59.902 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.35714285714285715
# 2023-10-10 10:33:08.571 | INFO     | __main__:training_model:210 - 
# Epoch 3
# 2023-10-10 10:33:08.572 | INFO     | __main__:training_model:214 - Training loss: 0.6749535351991653
# 2023-10-10 10:33:10.003 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.6873311400413513, [[ 0.11063756  0.16893572]
 
# 2023-10-10 10:33:10.007 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.5342465753424658
# 2023-10-10 10:33:10.009 | INFO     | __main__:training_model:220 - Validation loss: 0.6873311400413513
# 2023-10-10 10:33:10.010 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.5342465753424658
# 2023-10-10 10:33:18.707 | INFO     | __main__:training_model:210 - 
# Epoch 4
# 2023-10-10 10:33:18.708 | INFO     | __main__:training_model:214 - Training loss: 0.6750572621822357
# 2023-10-10 10:33:20.147 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.6758909722169241, [[ 0.082546    0.22458252]
 
# 2023-10-10 10:33:20.150 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.6101694915254238
# 2023-10-10 10:33:20.155 | INFO     | __main__:training_model:220 - Validation loss: 0.6758909722169241
# 2023-10-10 10:33:20.156 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.6101694915254238
# 2023-10-10 10:33:28.937 | INFO     | __main__:training_model:210 - 
# Epoch 5
# 2023-10-10 10:33:28.938 | INFO     | __main__:training_model:214 - Training loss: 0.661759041249752
# 2023-10-10 10:33:30.380 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.6659130950768789, [[ 0.07391369  0.2572748 ]
 
# 2023-10-10 10:33:30.383 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.623529411764706
# 2023-10-10 10:33:30.385 | INFO     | __main__:training_model:220 - Validation loss: 0.6659130950768789
# 2023-10-10 10:33:30.385 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.623529411764706
# 2023-10-10 10:33:33.174 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.6659130950768789, [[ 0.07391369  0.2572748 ]
 
# 2023-10-10 10:33:33.176 | INFO     | __main__:accuracy_per_class:101 - Class: 0
# 2023-10-10 10:33:33.177 | INFO     | __main__:accuracy_per_class:102 - Accuracy: 40/79
# 2023-10-10 10:33:33.177 | INFO     | __main__:accuracy_per_class:103 - % Accuracy: 0.5063291139240507

# 2023-10-10 10:33:33.178 | INFO     | __main__:accuracy_per_class:101 - Class: 1
# 2023-10-10 10:33:33.179 | INFO     | __main__:accuracy_per_class:102 - Accuracy: 53/78
# 2023-10-10 10:33:33.179 | INFO     | __main__:accuracy_per_class:103 - % Accuracy: 0.6794871794871795

# 2023-10-10 10:33:33.187 | INFO     | __main__:results:157 - {'0': {'precision': 0.6153846153846154, 'recall': 0.5063291139240507, 'f1-score': 0.5555555555555556, 'support': 79}, '1': {'precision': 0.5760869565217391, 'recall': 0.6794871794871795, 'f1-score': 0.623529411764706, 'support': 78}, 'accuracy': 0.5923566878980892, 'macro avg': {'precision': 0.5957357859531773, 'recall': 0.5929081467056151, 'f1-score': 0.5895424836601308, 'support': 157}, 'weighted avg': {'precision': 0.5958609377329953, 'recall': 0.5923566878980892, 'f1-score': 0.5893260064110571, 'support': 157}}
# 2023-10-10 10:33:33.189 | INFO     | __main__:call_setup:260 - Metric Report Call Setup: {'0': {'precision': 0.6153846153846154, 'recall': 0.5063291139240507, 'f1-score': 0.5555555555555556, 'support': 79}, '1': {'precision': 0.5760869565217391, 'recall': 0.6794871794871795, 'f1-score': 0.623529411764706, 'support': 78}, 'accuracy': 0.5923566878980892, 'macro avg': {'precision': 0.5957357859531773, 'recall': 0.5929081467056151, 'f1-score': 0.5895424836601308, 'support': 157}, 'weighted avg': {'precision': 0.5958609377329953, 'recall': 0.5923566878980892, 'f1-score': 0.5893260064110571, 'support': 157}}
# 2023-10-10 10:33:33.192 | DEBUG    | __main__:<module>:2 - Department: DPT_2
# 2023-10-10 10:33:33.510 | DEBUG    | __main__:<module>:37 - Department count samples: label                   104
# department              104
# entry_date              104
# general_id              104
# normative_identifier    104
# publication_date        104
# regulatory_authority    104
# subject                 104
# subject_length          104
# subject_unique_words    104
# subject_words           104
# content                 104
# text_length             104
# text_unique_words       104
# text_words              104
# type                    104
# unique_document_id      104
# url                     104
# new_content             104
# dtype: int64
# 2023-10-10 10:33:33.513 | DEBUG    | __main__:<module>:38 - Department count samples class 1: label                   52
# department              52
# entry_date              52
# general_id              52
# normative_identifier    52
# publication_date        52
# regulatory_authority    52
# subject                 52
# subject_length          52
# subject_unique_words    52
# subject_words           52
# content                 52
# text_length             52
# text_unique_words       52
# text_words              52
# type                    52
# unique_document_id      52
# url                     52
# new_content             52
# dtype: int64
# 2023-10-10 10:33:33.516 | DEBUG    | __main__:<module>:39 - Department count samples class 0: label                   52
# department              52
# entry_date              52
# general_id              52
# normative_identifier    52
# publication_date        52
# regulatory_authority    52
# subject                 52
# subject_length          52
# subject_unique_words    52
# subject_words           52
# content                 52
# text_length             52
# text_unique_words       52
# text_words              52
# type                    52
# unique_document_id      52
# url                     52
# new_content             52
# dtype: int64
# 2023-10-10 10:33:43.032 | INFO     | __main__:training_model:210 - 
# Epoch 1
# 2023-10-10 10:33:43.033 | INFO     | __main__:training_model:214 - Training loss: 0.6881098747253418
# 2023-10-10 10:33:43.424 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.6997461318969727, [[ 0.30694127  0.08413929]
 
# 2023-10-10 10:33:43.427 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.0909090909090909
# 2023-10-10 10:33:43.429 | INFO     | __main__:training_model:220 - Validation loss: 0.6997461318969727
# 2023-10-10 10:33:43.430 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.0909090909090909
# 2023-10-10 10:33:47.212 | INFO     | __main__:training_model:210 - 
# Epoch 2
# 2023-10-10 10:33:47.214 | INFO     | __main__:training_model:214 - Training loss: 0.6941970884799957
# 2023-10-10 10:33:47.606 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.6986801624298096, [[ 0.30528542  0.08728919]
 
# 2023-10-10 10:33:47.609 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.17391304347826084
# 2023-10-10 10:33:47.611 | INFO     | __main__:training_model:220 - Validation loss: 0.6986801624298096
# 2023-10-10 10:33:47.612 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.17391304347826084
# 2023-10-10 10:33:51.739 | INFO     | __main__:training_model:210 - 
# Epoch 3
# 2023-10-10 10:33:51.741 | INFO     | __main__:training_model:214 - Training loss: 0.69303297996521
# 2023-10-10 10:33:52.131 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.6972529292106628, [[ 0.30319765  0.09119336]
 
# 2023-10-10 10:33:52.134 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.17391304347826084
# 2023-10-10 10:33:52.135 | INFO     | __main__:training_model:220 - Validation loss: 0.6972529292106628
# 2023-10-10 10:33:52.136 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.17391304347826084
# 2023-10-10 10:33:56.043 | INFO     | __main__:training_model:210 - 
# Epoch 4
# 2023-10-10 10:33:56.045 | INFO     | __main__:training_model:214 - Training loss: 0.692831426858902
# 2023-10-10 10:33:56.436 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.6951631903648376, [[ 0.30027413  0.09689736]
 
# 2023-10-10 10:33:56.439 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.17391304347826084
# 2023-10-10 10:33:56.441 | INFO     | __main__:training_model:220 - Validation loss: 0.6951631903648376
# 2023-10-10 10:33:56.442 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.17391304347826084
# 2023-10-10 10:34:00.359 | INFO     | __main__:training_model:210 - 
# Epoch 5
# 2023-10-10 10:34:00.361 | INFO     | __main__:training_model:214 - Training loss: 0.6801402270793915
# 2023-10-10 10:34:00.751 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.6930978894233704, [[ 0.29772893  0.10173518]
 
# 2023-10-10 10:34:00.754 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.3076923076923077
# 2023-10-10 10:34:00.755 | INFO     | __main__:training_model:220 - Validation loss: 0.6930978894233704
# 2023-10-10 10:34:00.756 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.3076923076923077
# 2023-10-10 10:34:02.472 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.6930978894233704, [[ 0.29772893  0.10173518]
 
# 2023-10-10 10:34:02.473 | INFO     | __main__:accuracy_per_class:101 - Class: 0
# 2023-10-10 10:34:02.474 | INFO     | __main__:accuracy_per_class:102 - Accuracy: 20/21
# 2023-10-10 10:34:02.475 | INFO     | __main__:accuracy_per_class:103 - % Accuracy: 0.9523809523809523

# 2023-10-10 10:34:02.476 | INFO     | __main__:accuracy_per_class:101 - Class: 1
# 2023-10-10 10:34:02.476 | INFO     | __main__:accuracy_per_class:102 - Accuracy: 4/21
# 2023-10-10 10:34:02.477 | INFO     | __main__:accuracy_per_class:103 - % Accuracy: 0.19047619047619047

# 2023-10-10 10:34:02.484 | INFO     | __main__:results:157 - {'0': {'precision': 0.5405405405405406, 'recall': 0.9523809523809523, 'f1-score': 0.6896551724137931, 'support': 21}, '1': {'precision': 0.8, 'recall': 0.19047619047619047, 'f1-score': 0.3076923076923077, 'support': 21}, 'accuracy': 0.5714285714285714, 'macro avg': {'precision': 0.6702702702702703, 'recall': 0.5714285714285714, 'f1-score': 0.49867374005305043, 'support': 42}, 'weighted avg': {'precision': 0.6702702702702703, 'recall': 0.5714285714285714, 'f1-score': 0.49867374005305043, 'support': 42}}
# 2023-10-10 10:34:02.486 | INFO     | __main__:call_setup:260 - Metric Report Call Setup: {'0': {'precision': 0.5405405405405406, 'recall': 0.9523809523809523, 'f1-score': 0.6896551724137931, 'support': 21}, '1': {'precision': 0.8, 'recall': 0.19047619047619047, 'f1-score': 0.3076923076923077, 'support': 21}, 'accuracy': 0.5714285714285714, 'macro avg': {'precision': 0.6702702702702703, 'recall': 0.5714285714285714, 'f1-score': 0.49867374005305043, 'support': 42}, 'weighted avg': {'precision': 0.6702702702702703, 'recall': 0.5714285714285714, 'f1-score': 0.49867374005305043, 'support': 42}}

